In [2]:
from typing import List, Dict, Set, Any, Optional, Tuple, Literal, Callable
import time

import numpy as np
import jax
import jax.numpy as jnp
import jax.lax as lax
from jaxtyping import Array, Float, Int, PRNGKeyArray
import aeon
import pandas as pd

from features.sig_trp import SigVanillaTensorizedRandProj, SigRBFTensorizedRandProj
from features.sig import SigTransform, LogSigTransform
from features.base import TimeseriesFeatureTransformer, TabularTimeseriesFeatures, RandomGuesser
from features.sig_neural import RandomizedSignature
from utils import print_name, print_shape

from preprocessing.timeseries_augmentation import normalize_mean_std_traindata, normalize_streams, augment_time, add_basepoint_zero
from aeon.classification.sklearn import RotationForestClassifier
from sklearn.metrics import accuracy_score

jax.config.update('jax_platform_name', 'gpu') # Used to set the platform (cpu, gpu, etc.)
np.set_printoptions(precision=3, threshold=5) # Print options

from aeon.datasets.tsc_datasets import multivariate, univariate, univariate_equal_length
from aeon.datasets import load_classification

from run_univariate import do_experiments


In [ ]:
#if __name__ == "__main__":
#run experiments
d_res, d_meta, d_failed = do_experiments(list(univariate_equal_length))

# make dict of results
model_names = d_res[list(d_res.keys())[0]][0][0]
alpha_names = ["alpha_" + model_name for model_name in model_names]
df_accs = pd.DataFrame({dataset : accs for dataset, ((model_names, accs, alphas),) in d_res.items()}).transpose()
df_accs.columns = model_names
df_alphas = pd.DataFrame({dataset : alphas for dataset, ((model_names, accs, alphas),) in d_res.items()}).transpose()
df_alphas.columns = alpha_names
meta = pd.DataFrame(d_meta).transpose()

df_accs = pd.concat([meta, df_accs], axis=1)
df_alphas = pd.concat([meta, df_alphas], axis=1)

# save
df_accs.to_pickle("df_accs_univariateTSC.pkl")
df_accs.to_pickle("df_alphas_univariateTSC.pkl")
print(d_failed)

In [3]:
df_accs = pd.read_pickle("df_accs_univariateTSC.pkl")
df_accs


,N_train,N_test,T,D,Random Guesser,Tabular,Sig,Log Sig,Sig Vanilla TRP,Sig RBF TRP,Randomized Signature,Rocket,MiniRocket,MultiRocket
Yoga,300,3000,426,1,0.507333,0.630000,0.662667,0.579667,0.649333,0.778000,0.658000,0.849667,0.839000,0.830333
SonyAIBORobotSurface1,20,601,70,1,0.434276,0.735441,0.707155,0.710483,0.866889,0.831947,0.514143,0.785358,0.871880,0.918469
MoteStrain,20,1252,84,1,0.482428,0.882588,0.654153,0.650160,0.674121,0.706869,0.410543,0.813099,0.833866,0.886581
ArrowHead,36,175,251,1,0.337143,0.617143,0.537143,0.520000,0.491429,0.480000,0.320000,0.725714,0.794286,0.828571
ECG5000,500,4500,140,1,0.339778,0.928222,0.916667,0.906222,0.908222,0.830667,0.494667,0.936000,0.934000,0.931111
NonInvasiveFetalECGThorax1,1800,1965,750,1,0.022901,0.707888,0.508906,0.350636,0.525191,0.704835,0.097710,0.855471,0.875827,0.808651
DistalPhalanxTW,400,139,80,1,0.316547,0.611511,0.669065,0.633094,0.589928,0.489209,0.410072,0.611511,0.625899,0.561151
TwoLeadECG,23,1139,82,1,0.500439,0.941176,0.629500,0.794557,0.597015,0.611062,0.482002,0.854258,0.838455,0.871817
Ham,109,105,431,1,0.514286,0.561905,0.619048,0.609524,0.523810,0.476190,0.590476,0.666667,0.619048,0.619048
DiatomSizeReduction,16,306,345,1,0.264706,0.964052,0.898693,0.869281,0.709150,0.800654,0.284314,0.950980,0.954248,0.957516


In [ ]:
df_accs.mean()

In [ ]:
N_train = 200
df_accs[df_accs["N_train"] > N_train]

In [ ]:
df_accs[df_accs["N_train"] > N_train].mean()

# new below

In [ ]:
def get_aeon_dataset(
        dataset_name:str, 
        extract_path = "/home/nikita/hdd/Data/TSC/"
        ):
    """Loads a dataset from the UCR/UEA archive using 
    the aeon library.

    Args:
        dataset_name (str): Name of the dataset

    Returns:
        Tuple: 4-tuple of the form (X_train, y_train, X_test, y_test)
    """
    X_train, y_train = load_classification(dataset_name, split="train", extract_path=extract_path)
    X_test, y_test = load_classification(dataset_name, split="test", extract_path=extract_path)

    return X_train.transpose(0,2,1), y_train, X_test.transpose(0,2,1), y_test


get_aeon_dataset("NonInvasiveFetalECGThorax2")

